<a href="https://colab.research.google.com/github/Yyyyyjkim/review_analysis/blob/master/1_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt-get update
!apt install chromium-chromedriver
!pip install requests
!pip install bs4
!pip install selenium

In [0]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [0]:
class crawling:
  def __init__(self):
    self.options = webdriver.ChromeOptions()
    self.options.add_argument('--headless')
    self.options.add_argument('--no-sandbox')
    self.options.add_argument('--disable-dev-shm-usage')
    self.driver = webdriver.Chrome('chromedriver',options=self.options)
  
  def get_text(self,url,sorting,pagenum):
    # go to review page and sorted by sorting
    self.driver.get(url)
    self.driver.find_element_by_xpath("//div[@class='detail_tab_floatable']//li[@class='item']/a[@class='link N=a:tab.review']").click()
    time.sleep(5)
    div_class = 'header_review _review_list_header'
    ul_class = 'sort_list _review_list_header_sort'
    path = f"//div[@class='{div_class}']//ul[@class='{ul_class}']//a[text()='{sorting}']"
    self.driver.find_element_by_xpath(path).click()
    time.sleep(5)

    # get text from 1 to pagenum
    html = self.driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    review_page = soup.find_all('p',class_='review_text _review_text')
    review = [each_line.get_text().strip() for each_line in review_page[:20]]
    for page in range(2,(pagenum+1)):
        path_page = f"//nav[@class='module_pagination _review_list_page']//a[text()='{page}']"
        self.driver.find_element_by_xpath(path_page).click()
        time.sleep(5)
        html = self.driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_page = soup.find_all('p',class_='review_text _review_text')
        review_page = [each_line.get_text().strip() for each_line in review_page[:20]]
        review.extend(review_page)
    
    return review

In [0]:
crawl = crawling()
text_rank =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","랭킹순",10)
# text_date =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","최신순",10)
# text_desc =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","평점 높은순",10)
# text_asc =crawl.get_text("https://smartstore.naver.com/jainystore/products/2455969597","평점 낮은순",10)

In [0]:
import pickle
import pandas as pd

In [0]:
with open('/content/drive/My Drive/comment_analysis/data/crawling.pickle','wb') as f:
  pickle.dump(crawling,f)